# Process Optimization
One of the main features of **CADET-Process** is process optimization. 

an `OptimizationProblem` class is introduced that decouples the problem formulation from the `Optimizer` used for its solution (see overview), allowing for a simple comparison of different optimization approaches.
The `OptimizationVariables` $x$ may refer to any attribute of the `ProcessModel`.
This includes model parameters, as well as `FlowSheet`-events.
As for the latter, not only the time when they are executed can be optimized, but also the value to which the attribute is changed when the `Event` is performed, allowing for structural optimization.
Bound constraints and linear constraints can limit the parameter space and the user is free to define arbitrary function $f(x)$ as the objective, and add arbitrary nonlinear constraint functions $g(x)$ to the `OptimizationProblem`.

The `OptimizerAdapter` provides a unified interface for using external optimization libraries.
It is responsible for converting the `OptimizationProblem` to the specific API of the external `Optimizer`.
Currently, adapters to [pymoo](https://pymoo.org/) and [SciPy's optimization suite](https://docs.scipy.org/doc/scipy/tutorial/optimize.html) are implemented, all of which are published under open source licenses that allow for academic as well as commercial use.

## Optimization Problem
After import, the `OptimizationProblem` is initialized with a name.

In [1]:
from CADETProcess.optimization import OptimizationProblem

optimization_problem = OptimizationProblem(name='batch elution')

Then, one or more `EvaluationObjects` can be added to the problem.
These are objects which contain the parameters which should be optimized (e.g. a `Process` or a `Fractionator`).

For this demonstration, the process model from a previous tutorial is used.

In [2]:
import sys
sys.path.insert(0, '../../_templates')
from batch_elution import process

In [3]:
optimization_problem.add_evaluation_object(process)

In this example, we want to find the optimal cycle time and feed duration.
To specify these variable, the `add_variable` method is used.
First, it takes the path to the variable in the evaluation object.
Lower and upper bounds can be specified and it is possible to specify with which `EvaluationObject` the variable is associated.
By default, the variable is associated with all evaluation objects.

In [4]:
optimization_problem.add_variable('cycle_time', lb=10, ub=600, evaluation_objects=process)
optimization_problem.add_variable('feed_duration.time', lb=10, ub=300)

Moreover, linear constraints can be added to the optimization problem.
In this example, it needs to be ensured that the cycle time is always greater than the feed duration.
Linear constraints are usually defined in the following way

$$
A \cdot x \leq b
$$

In **CADET-Process**, add each row $a$ of the constraint matrix needs to be added individually.
The `add_linear_constraint` function takes the variables subject to the constraint as first argument.
The left-hand side $a$ and the bound $b_a$ are passed as second and third argument. 
It is important to note that the column order in $a$ is inferred from the order in which the optimization variables are passed.

In [5]:
optimization_problem.add_linear_constraint(['feed_duration.time', 'cycle_time'], [1,-1], 0)

## Evaluation Toolchain
In many situations, some preprocessing steps are required before the objective function can be evaluated.

In the current example, to evaluate the performance of the process, the following steps need to be performed:
- Simulate the process until stationarity is reached.
- Determine fractionation times under purity constraints.
- Calculate objective functions; Here, two objectives are considered:
	- Productivity,
	- Yield recovery.

To implement these evaluation toolchains, **CADET-Process** provides a mechanism to add `Evaluators` to an `OptimizationProblem` which can be referenced by objective and constraint functions.
Any callable function can be added as `Evaluator`, assuming the first argument is the result of the previous step and it returns a single result object which is then processed by the next step.
Additional arguments and keyword arguments can be passed using `args` and `kwargs` when adding the `Evaluator`.
Optionally, the intermediate results can be cached when different objective and constraint functions require the same preprocessing steps.

To demonstrate this, a `ProcessSimulator` and a fully configured `FractionationOptimizer` are added to the `OptimizationProblem`.
First, `CADET` is configured as usual, and the assertion of cyclic stationarity is enabled.

In [6]:
from CADETProcess.simulator import Cadet
process_simulator = Cadet()
process_simulator.evaluate_stationarity = True

optimization_problem.add_evaluator(process_simulator)

Note that storing all simulation results requires large amounts of disk space.
Here, only the fractionation results are cached since they will be used by both objectives.

In [7]:
from CADETProcess.fractionation import FractionationOptimizer
fractionation_optimization = FractionationOptimizer()

optimization_problem.add_evaluator(fractionation_optimization, cache=True, kwargs={'purity_required': [0.95, 0.95]})

When adding the objectives (or nonlinear constraint) functions, the evaluators are added as requirements.

In [8]:
from CADETProcess.performance import Recovery, Productivity

recovery = Recovery()
optimization_problem.add_objective(recovery, n_objectives=2, requires=[process_simulator, fractionation_optimization])
productivity = Productivity()
optimization_problem.add_objective(productivity, n_objectives=2, requires=[process_simulator, fractionation_optimization])

Now, when the objectives are evaluated, the process is only simulated once, and the optimal fractionation times only need to be determined once.

In [9]:
f = optimization_problem.evaluate_objectives([300, 60])
print(f"objectives: {f}")

objectives: [-0.8323210414681842, -0.7199629648823258, -0.020442593825471585, -0.017682972948166478]


## Optimizer
Before the optimization can be run, the `Optimizer` needs to be initialized and configured.
For this example, `U_NSGA3` is used, a genetic algorithm {cite}`Seada2016`.
Important parameters are the population size (`pop_size`), the maximum number of generations (`n_max_gen`) and the number of cores (`n_cores`) which can be used during optimization.

In [10]:
from CADETProcess.optimization import U_NSGA3

optimizer = U_NSGA3()
optimizer.pop_size = 4
optimizer.n_max_gen = 4
optimizer.n_cores = 1

To start the simulation, the `OptimizationProblem` needs to be passed to the `optimize()` method.
**CADET-Process** automatically stores checkpoints so that an optimization can be interrupted and restarted.
To load from an existing file, set `use_checkpoint=True`.

<!-- ``` -->

In [12]:
optimization_results = optimizer.optimize(
    optimization_problem,
    use_checkpoint=True,
)

Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled


Simulation of batch elution with parameters {'parameters': {'feed_on': {'time': 0.0, 'state': array([1.e-06, 0.e+00, 0.e+00, 0.e+00])}, 'feed_duration': {'time': 73.48978572857229}, 'cycle_time': 116.25863904323063, 'flow_sheet': {'feed': {'flow_rate': array([0., 0., 0., 0.]), 'c': array([[10.,  0.,  0.,  0.],
       [10.,  0.,  0.,  0.]])}, 'output_states': {'feed': [1], 'eluent': [1], 'column': [1], 'outlet': []}, 'eluent': {'flow_rate': array([0., 0., 0., 0.]), 'c': array([[0., 0., 0., 0.],
       [0., 0., 0., 0.]])}, 'column': {'length': 0.6, 'diameter': 0.024, 'axial_dispersion': 4.7e-07, 'flow_direction': 1, 'total_porosity': 0.7, 'binding_model': {'is_kinetic': False, 'adsorption_rate': [0.02, 0.03], 'desorption_rate': [1, 1], 'capacity': [100, 100]}, 'discretization': {'ncol': 100, 'use_analytic_jacobian': True, 'reconstruction': 'WENO', 'weno': {'boundary_model': 0, 'weno_eps': 1e-10, 'weno_order': 3}, 'consistency_solver': {'solver_name': 'LEVMAR', 'init_damping': 0.01, 'min_

n_gen |  n_eval |  n_nds  |     eps      |  indicator  
    1 |       4 |       1 |            - |            -


/opt/tljh/user/lib/python3.9/site-packages/pymoo/core/plot.py:133: UserWarning: Tight layout not applied. tight_layout cannot make axes width small enough to accommodate all axes decorations
  self.fig.tight_layout()
Simulation of batch elution with parameters {'parameters': {'feed_on': {'time': 0.0, 'state': array([1.e-06, 0.e+00, 0.e+00, 0.e+00])}, 'feed_duration': {'time': 79.45630536446274}, 'cycle_time': 116.25863904323063, 'flow_sheet': {'feed': {'flow_rate': array([0., 0., 0., 0.]), 'c': array([[10.,  0.,  0.,  0.],
       [10.,  0.,  0.,  0.]])}, 'output_states': {'feed': [1], 'eluent': [1], 'column': [1], 'outlet': []}, 'eluent': {'flow_rate': array([0., 0., 0., 0.]), 'c': array([[0., 0., 0., 0.],
       [0., 0., 0., 0.]])}, 'column': {'length': 0.6, 'diameter': 0.024, 'axial_dispersion': 4.7e-07, 'flow_direction': 1, 'total_porosity': 0.7, 'binding_model': {'is_kinetic': False, 'adsorption_rate': [0.02, 0.03], 'desorption_rate': [1, 1], 'capacity': [100, 100]}, 'discretizatio

    2 |       8 |       1 |  0.00000E+00 |            f


/opt/tljh/user/lib/python3.9/site-packages/pymoo/core/plot.py:133: UserWarning: Tight layout not applied. tight_layout cannot make axes width small enough to accommodate all axes decorations
  self.fig.tight_layout()
Simulation of batch elution with parameters {'parameters': {'feed_on': {'time': 0.0, 'state': array([1.e-06, 0.e+00, 0.e+00, 0.e+00])}, 'feed_duration': {'time': 176.27146259965392}, 'cycle_time': 361.8746679955175, 'flow_sheet': {'feed': {'flow_rate': array([0., 0., 0., 0.]), 'c': array([[10.,  0.,  0.,  0.],
       [10.,  0.,  0.,  0.]])}, 'output_states': {'feed': [1], 'eluent': [1], 'column': [1], 'outlet': []}, 'eluent': {'flow_rate': array([0., 0., 0., 0.]), 'c': array([[0., 0., 0., 0.],
       [0., 0., 0., 0.]])}, 'column': {'length': 0.6, 'diameter': 0.024, 'axial_dispersion': 4.7e-07, 'flow_direction': 1, 'total_porosity': 0.7, 'binding_model': {'is_kinetic': False, 'adsorption_rate': [0.02, 0.03], 'desorption_rate': [1, 1], 'capacity': [100, 100]}, 'discretizatio

    3 |      12 |       1 |  0.00000E+00 |            f


/opt/tljh/user/lib/python3.9/site-packages/pymoo/core/plot.py:133: UserWarning: Tight layout not applied. tight_layout cannot make axes width small enough to accommodate all axes decorations
  self.fig.tight_layout()
There was an exception in simulate_to_stationarity
Traceback (most recent call last):
  File "/opt/tljh/user/lib/python3.9/site-packages/CADETProcess/log.py", line 102, in wrapper
    return function(*args, **kwargs)
  File "/opt/tljh/user/lib/python3.9/site-packages/CADETProcess/simulator/simulator.py", line 280, in simulate_to_stationarity
    results = self.run(process, **kwargs)
  File "/opt/tljh/user/lib/python3.9/site-packages/CADETProcess/simulator/cadetAdapter.py", line 259, in run
    return_information = cadet.run_load(timeout=self.timeout)
  File "/opt/tljh/user/lib/python3.9/site-packages/cadet/cadet.py", line 218, in run_load
    data = self.cadet_runner.run(simulation=self.root.input, filename=self.filename, timeout=timeout, check=check)
  File "/opt/tljh/use

KeyboardInterrupt: 

In the results, the information about the optimization results are stored such as best individual etc.